In [116]:
import pennylane as qml
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from pennylane import numpy as np
from copy import deepcopy

In [28]:
data = datasets.load_iris()
X, y = data["data"][:100], data["target"][:100]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

In [113]:
wires = 4
dev = qml.device("lightning.qubit", wires=wires)

# maps the classical data to quantum features
def feature_map(features):
    for i, xi in enumerate(features):
        qml.RX(xi, wires=i)

# the trainable part of the circuit
def variational_form(theta):
    for i in range(wires-1):
        qml.CNOT(wires=[i, i+1])
    qml.CNOT(wires=[wires-1, 0])
    for i in range(wires):
        qml.RY(theta[i], wires=i)


theta = np.random.randn(4)

# full qnantum circuit
@qml.qnode(dev)
def circuit(theta, input=None):
    feature_map(input)
    variational_form(theta)
    return qml.expval(qml.Z(0))

drawer = qml.draw(circuit)
print(drawer(theta, X_train[1]))
print(circuit(theta, X_train[0]))

0: ──RX(5.60)─╭●───────╭X──RY(0.10)──┤  <Z>
1: ──RX(3.00)─╰X─╭●────│───RY(0.98)──┤     
2: ──RX(4.10)────╰X─╭●─│───RY(-0.93)─┤     
3: ──RX(1.30)───────╰X─╰●──RY(-1.09)─┤     
0.052876705316244714


In [ ]:
def train(epoch=20):
    """
    https://pennylane.ai/blog/2022/06/how-to-choose-your-optimizer/
    https://docs.pennylane.ai/en/stable/code/api/pennylane.GradientDescentOptimizer.html
    """
    theta = np.random.randn(4, requires_grad=True)
    optimizer = qml.GradientDescentOptimizer()
    
    for e in range(epoch):
        epoch_loss = []
        for i in zip(X_train, y_train):
            y_true = np.array([i[1]])
            # y_pred = circuit(theta, i[0])
            # y_pred = np.array([y_pred])
            # loss = log_loss(y_true, y_pred, labels=[0, 1])

            def obj_fn(opt_theta):
                y_pred = circuit(opt_theta, i[0])
                y_pred = np.array([y_pred._value])
                loss = log_loss(y_true, y_pred, labels=[0, 1])
                return loss
            # theta = optimizer.step(obj_fn, (theta,), kwargs={})
            theta, cost = optimizer.step_and_cost(obj_fn, theta)
            epoch_loss.append(cost)
        print(np.mean(epoch_loss))
train()

In [117]:
def loss(a, b):
    return (a-b)**2

def gradient(X, y, theta):
    y_pred = circuit(theta, X)
    delta = 0.01
    grad = []
    pred = circuit(theta, X)

    for i in range(len(theta)):
        dtheta = deepcopy(theta)
        dtheta[i] += delta
        dpred = circuit(dtheta, X)

        grad.append((loss(pred, y)-loss(dpred, y))/delta)
    return np.array(grad)


        
    


def train(epoch=20):
    eta = 0.05
    theta = np.random.randn(4, requires_grad=True)
    
    for e in range(epoch):
        epoch_loss = []
        for i in zip(X_train, y_train):
            y_true = i[1]
            y_pred = circuit(theta, i[0])
            epoch_loss.append(loss(y_true, y_pred))
            theta = theta - eta * gradient(i[0], i[1], theta)
            
        print(np.mean(epoch_loss))
train()

0.49389717727045984
0.5568980214852572
0.6293233879923873
0.6696755820288722
0.6820028157106398
0.6848390364016292
0.6854288412128536
0.6855428845198575
0.6855617643927869
0.6855633231304518
0.6855625295221713
0.6855618441465096
0.6855614608832422
0.6855612701169977
0.685561179271649
0.6855611368144354
0.6855611171364766
0.685561108050552
0.6855611038625276
0.6855611019336453
